<a href="https://colab.research.google.com/github/MrsIgnis/MOCI/blob/main/MOCI_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install nltk pymorphy3

In [64]:
import re
import nltk
import pymorphy3
import math
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [65]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
stop_words_ru = set(stopwords.words("russian"))
stop_words_en = set(stopwords.words("english"))

In [67]:
lemma_ru = pymorphy3.MorphAnalyzer()
lemma_en = WordNetLemmatizer()

In [68]:
file_paths = ['/content/test_1.txt', '/content/test_2.txt', '/content/test_3.txt']
texts = [open(path, 'r', encoding='utf-8').read() for path in file_paths]
text_1, text_2, text_3 = texts
print(text_1, text_2, text_3, sep='\n' + '-' * 30 + '\n')

Когда скромняга бард отдыхал от дел
С Геральтом из Ривии он песню эту пел.
Сразился Белый Волк с велиречивым чертом
Эльфов покромсал несчетные когорты.
------------------------------
Целился тот черт мне рогом прямо в глаз
И тут Ведьмак крикнул «Вот твой смертный час!»

Ведьмаку заплатите чеканной монетой, чеканной монетой,
Ведьмаку заплатите – зачтется все это вам!
------------------------------
Он эльфов всех прогнал за дальний перевал
Высокие горы на вечный привал

Он бьет не в бровь, а в глаз, был ранен много раз
Он людям товарищ всегда он за нас


**I. Применить к текстам лемматизацию, удаление стоп слов и токенизацию по словам**

In [69]:
def is_russian(word: str) -> bool:
    return bool(re.search('[а-яё]', word, re.IGNORECASE))

def is_english(word: str) -> bool:
    return bool(re.search('[a-z]', word, re.IGNORECASE))

In [70]:
def preprocessing(texts: list[str]) -> list[list[str]]:
    processed_texts = []

    for text in texts:
        text = text.lower()
        text = re.sub(r'[^а-яёa-z\s]', '', text, flags=re.IGNORECASE)
        words = word_tokenize(text)
        processed_words = []

        for word in words:
            if word not in stop_words_ru and word not in stop_words_en:
                if is_russian(word):
                    lemma = lemma_ru.parse(word)[0].normal_form
                elif is_english(word):
                    lemma = lemma_en.lemmatize(word)
                else:
                    lemma = word
                processed_words.append(lemma)

        processed_texts.append(processed_words)

    return processed_texts

In [71]:
processed_texts = preprocessing(texts)
for i, processed_text in enumerate(processed_texts, start=1):
    print(f"Обработанный текст {i}:")
    print(processed_text)
    print('-' * 30)

Обработанный текст 1:
['скромняга', 'бард', 'отдыхать', 'дело', 'геральт', 'ривия', 'песня', 'петь', 'сразиться', 'белый', 'волк', 'велиречивый', 'чёрт', 'эльф', 'покромсать', 'несчётный', 'когорта']
------------------------------
Обработанный текст 2:
['целиться', 'черта', 'рог', 'прямо', 'глаз', 'ведьмак', 'крикнуть', 'твой', 'смертный', 'час', 'ведьмак', 'заплатить', 'чеканный', 'монета', 'чеканный', 'монета', 'ведьмак', 'заплатить', 'зачесться', 'это']
------------------------------
Обработанный текст 3:
['эльф', 'прогнать', 'дальний', 'перевал', 'высокий', 'гора', 'вечный', 'привал', 'бить', 'бровь', 'глаз', 'ранить', 'человек', 'товарищ']
------------------------------


In [72]:
preprocessed_documents = []
for i in processed_texts:
    preprocessed_documents.append(i)
print("Обработанные тексты в одном списке:", preprocessed_documents, sep='\n')

Обработанные тексты в одном списке:
[['скромняга', 'бард', 'отдыхать', 'дело', 'геральт', 'ривия', 'песня', 'петь', 'сразиться', 'белый', 'волк', 'велиречивый', 'чёрт', 'эльф', 'покромсать', 'несчётный', 'когорта'], ['целиться', 'черта', 'рог', 'прямо', 'глаз', 'ведьмак', 'крикнуть', 'твой', 'смертный', 'час', 'ведьмак', 'заплатить', 'чеканный', 'монета', 'чеканный', 'монета', 'ведьмак', 'заплатить', 'зачесться', 'это'], ['эльф', 'прогнать', 'дальний', 'перевал', 'высокий', 'гора', 'вечный', 'привал', 'бить', 'бровь', 'глаз', 'ранить', 'человек', 'товарищ']]


**II. Реализовать Bag of Words**

In [73]:
def make_dict(texts: list[list[str]]) -> dict[str, int]:
    vocab = []
    for text in texts:
        for word in text:
            if word not in vocab:
                vocab.append(word)
    return vocab

In [74]:
unic_words = make_dict(processed_texts)
print("Словарь уникальных слов:", unic_words, sep='\n')

Словарь уникальных слов:
['скромняга', 'бард', 'отдыхать', 'дело', 'геральт', 'ривия', 'песня', 'петь', 'сразиться', 'белый', 'волк', 'велиречивый', 'чёрт', 'эльф', 'покромсать', 'несчётный', 'когорта', 'целиться', 'черта', 'рог', 'прямо', 'глаз', 'ведьмак', 'крикнуть', 'твой', 'смертный', 'час', 'заплатить', 'чеканный', 'монета', 'зачесться', 'это', 'прогнать', 'дальний', 'перевал', 'высокий', 'гора', 'вечный', 'привал', 'бить', 'бровь', 'ранить', 'человек', 'товарищ']


In [75]:
def Bag_of_Words(texts: list[list[str]], vocab: list[str]) -> list[list[int]]:
    bow = []

    for text in texts:
        word_count = [0] * len(vocab)

        for word in text:
            if word in vocab:
                index = vocab.index(word)
                word_count[index] += 1
        bow.append(word_count)

    return bow

In [76]:
bow_matrix = Bag_of_Words(processed_texts, unic_words)
print("Матрица Bag of Words:")
for row in bow_matrix:
    print(row)

Матрица Bag of Words:
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


**III. Реализовать TF-IDF**

In [77]:
def count_num_words(texts: list[list[str]]) -> list[int]:
    word_counts = []
    for text in texts:
        word_counts.append(len(text))
    return word_counts

In [78]:
word_counts = count_num_words(processed_texts)
print("Количество слов в каждом тексте:", word_counts)

Количество слов в каждом тексте: [17, 20, 14]


In [79]:
def compute_tf(bow_matrix: list[list[int]], word_counts: list[int]) -> list[list[float]]:
    tf_matrix = []

    for i, row in enumerate(bow_matrix):
        total_words = word_counts[i]
        tf_row = []

        for count in row:
            if total_words > 0:
                tf_row.append(count / total_words)
            else:
                tf_row.append(0)
        tf_matrix.append(tf_row)

    return tf_matrix

In [80]:
tf_matrix = compute_tf(bow_matrix, word_counts)
print("Матрица TF (Term Frequency):")
for row in tf_matrix:
    print(row)

Матрица TF (Term Frequency):
[0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.05, 0.05, 0.05, 0.15, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.05, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.071428571428

In [81]:
def compute_idf(texts: list[list[str]], vocab: list[str]) -> list[float]:
    idf = []
    num_docs = len(texts)

    for word in vocab:
        count = 0
        for text in texts:
            if word in text:
                count += 1
        idf.append(math.log((num_docs + 1) / (count + 1)) + 1)

    return idf

In [82]:
idf_values = compute_idf(processed_texts, unic_words)
print("Значения IDF (Inverse Document Frequency):")
print(idf_values)

Значения IDF (Inverse Document Frequency):
[1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.2876820724517808, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.2876820724517808, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454]


In [83]:
def compute_tfidf(tf_matrix: list[list[float]], idf_values: list[float]) -> list[list[float]]:
    tfidf_matrix = []

    for tf_row in tf_matrix:
        tfidf_row = []

        for i, tf_value in enumerate(tf_row):
            tfidf_row.append(tf_value * idf_values[i])
        tfidf_matrix.append(tfidf_row)

    return tfidf_matrix

In [84]:
tfidf_matrix = compute_tfidf(tf_matrix, idf_values)
print("Матрица TF-IDF:")
for row in tfidf_matrix:
    print(row)

Матрица TF-IDF:
[0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.07574600426186946, 0.09959689297411443, 0.09959689297411443, 0.09959689297411443, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08465735902799727, 0.08465735902799727, 0.08465735902799727, 0.08465735902799727, 0.06438410362258905, 0.2539720770839918, 0.08465735902799727, 0.08465735902799727, 0.08465735902799727, 0.08465735902799727, 0.16931471805599455, 0.16931471805599455, 0.16931471805599455, 0.08465735902799727, 0.08465735902799727, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 